In [1]:
#L'exécution de cette cellule permet de démarrer l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1669402518611_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1669402518611_0001,pyspark,idle,Link,Link,✔


In [3]:
import pandas as pd
import numpy as np

import io

import os

import sys

#os.environ['PYSPARK_PYTHON'] = sys.executable
#os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import tensorflow as tf

from PIL import Image

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model

from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql.types import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
tf.__version__

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2.4.1'

In [4]:
PATH = 's3://fahad-data-oc-p8'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+ PATH+'\nPATH_Data:   '+ PATH_Data+'\nPATH_Result: '+ PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://fahad-data-oc-p8
PATH_Data:   s3://fahad-data-oc-p8/Test
PATH_Result: s3://fahad-data-oc-p8/Results

In [5]:
images = spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg").option("recursiveFileLookup", "true").load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://fahad-data-o...|2022-11-07 20:37:43|  6328|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:37:19|  6322|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:40:22|  6308|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:39:10|  6304|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:39:59|  6300|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:39:23|  6291|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:38:50|  6280|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:39:07|  6277|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:37:14|  6269|[FF D8 FF E0 00 1...|
|s3://fahad-data-o...|2022-11-07 20:39:05|  6268|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only s

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'), -2))
print(images.printSchema())
print(images.select('path', 'label').show(10, False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+----------------------------------------------------------+--------------+
|path                                                      |label         |
+----------------------------------------------------------+--------------+
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/114_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/103_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/101_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/96_100.jpg |Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/100_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/121_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Test/Apple Golden 1/113_100.jpg|Apple Golden 1|
|s3://fahad-data-oc-p8/Test/Te

In [8]:
model = MobileNetV2(weights = 'imagenet', include_top = True, input_shape = (224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 0s 0us/step

In [9]:
new_model = Model(inputs = model.input, outputs = model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [12]:
def model_fn():
    '''Renvoie un modèle MobileNetV2 avec la couche supérieure supprimée
    et des poids broadcasted pré-entraînés.'''
    model = MobileNetV2(weights = 'imagenet', include_top = True, input_shape = (224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs = model.input, outputs = model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def preprocess(content):
    '''Prétraite les octets d'image bruts pour la prédiction.'''
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    '''Caractéristiques d'un pd.Series d'images brutes à l'aide du modèle d'entrée.
    retour : un pd.Série de fonctionnalités d'image.'''
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    #Pour certaines couches, les entités de sortie seront des tenseurs multidimensionnels.
    #Nous aplatissons caractéristiques des tenseurs en vecteurs pour un stockage plus facile en dataframes dans Spark.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#@pandas_udf("long")
#def plug_one(s: pd.Series) -> pd.Series:

In [16]:
#@pandas_udf(('array'), PandasUDFType.SCALAR_ITER)
#@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
#def featurize_udf(content_series_iter: pd.Series) -> pd.Series:
#@pandas_udf('long', PandasUDFType.SCALAR_ITER)
@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''Cette méthode est une UDF Scalar Iterator pandas enveloppant notre fonction de caractérisation.
    Le décorateur spécifie que cela renvoie une colonne Spark DataFrame de type ArrayType(FloatType).
    param content_series_iter : cet argument est un itérateur sur des lots de données, où chaque lot
    est une série de pandas de données d'image.'''
    #Avec les UDF pandas de Scalar Iterator, nous pouvons charger le modèle une fois puis le réutiliser pour plusieurs lots de données. Cela amortit les frais généraux liés au chargement de gros modèles.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
features_df = images.repartition(24).select(col("path"), col("label"), featurize_udf("content").alias("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://fahad-data-oc-p8/Results

In [19]:
features_df.write.mode('overwrite').parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#df = pd.read_parquet(PATH_Result, engine = 'pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df = spark.read.format('parquet').load(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(path='s3://fahad-data-oc-p8/Test/Test/Rambutan/63_100.jpg', label='Rambutan', features=[0.4528220295906067, 3.200425863265991, 0.0, 1.1370848417282104, 0.2181774079799652, 0.008455977775156498, 0.22968751192092896, 1.2391432523727417, 0.0, 0.0, 0.9595006704330444, 0.2623729705810547, 0.3823898136615753, 0.17860838770866394, 0.10032211989164352, 0.0, 0.06200098991394043, 0.057273250073194504, 0.0, 1.0711286067962646, 0.0, 0.0, 0.3108851909637451, 1.9870631694793701, 0.0, 0.02160801738500595, 1.2737057209014893, 0.2719417214393616, 0.8172907829284668, 0.0, 0.0, 0.0, 0.0, 4.4469733238220215, 0.0, 0.0, 0.8054088950157166, 0.033388469368219376, 0.016496052965521812, 0.0, 0.11240679770708084, 0.057882994413375854, 0.0, 0.0, 0.6904123425483704, 0.0038977714721113443, 0.28279468417167664, 1.180785894393921, 0.023461880162358284, 2.3759517669677734, 0.47417688369750977, 0.016826631501317024, 0.0006091998075135052, 0.0, 0.017355050891637802, 0.06920918822288513, 0.0, 0.5764712691307068, 0.16